In [194]:
import sklearn
import sklearn.datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn.metrics as metrics
import sklearn.naive_bayes
from sklearn.model_selection import train_test_split

import pandas as pd
from importlib import reload
from textblob import TextBlob

import preprocessing

import numpy as np

import scipy as sp

In [9]:
target_folder = 'Dataset'
train_data = pd.read_csv(target_folder + '/train.csv')
test_data = pd.read_csv(target_folder + '/test.csv')
sumbission_example = pd.read_csv(target_folder + '/sample_submission.csv')
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
5,0001ea8717f6de06,0.5,0.5,0.5,0.5,0.5,0.5
6,00024115d4cbde0f,0.5,0.5,0.5,0.5,0.5,0.5
7,000247e83dcc1211,0.5,0.5,0.5,0.5,0.5,0.5
8,00025358d4737918,0.5,0.5,0.5,0.5,0.5,0.5
9,00026d1092fe71cc,0.5,0.5,0.5,0.5,0.5,0.5


In [195]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
Xwo = vectorizer.fit_transform(train_data['comment_text'])
Xch = char_vectorizer.fit_transform(train_data['comment_text'])
X = sp.sparse.hstack([Xwo, Xch])
#X_test = vectorizer.transform(test_data['comment_text'])

In [178]:
yt = train_data['toxic']
yst = train_data['severe_toxic']
yo = train_data['obscene']
yth = train_data['threat']
yi = train_data['insult']
yih = train_data['identity_hate']

In [28]:
train_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [197]:
X_tr, X_te, yt_tr, yt_te, yst_tr, yst_te, yo_tr, yo_te, yth_tr, yth_te, yi_tr, yi_te, yih_tr, yih_te = train_test_split(X, yt, yst, yo, yth, yi, yih, test_size=0.33, random_state=42)

In [218]:
#clf = sklearn.linear_model.LogisticRegression(n_jobs=-1, verbose=2, solver='lbfgs', max_iter=15000)
clft = sklearn.naive_bayes.MultinomialNB(alpha=0.98)
clft.fit(X_tr, yt_tr)
predt = clft.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yt_te, predt))

clfst = sklearn.naive_bayes.MultinomialNB(alpha=0.22)
clfst.fit(X_tr, yst_tr)
predst = clfst.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yst_te, predst))

clfo = sklearn.naive_bayes.MultinomialNB(alpha=0.80)
clfo.fit(X_tr, yo_tr)
predo = clfo.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yo_te, predo))

clfth = sklearn.naive_bayes.MultinomialNB(alpha=.09)
clfth.fit(X_tr, yth_tr)
predth = clfth.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yth_te, predth))

clfi = sklearn.naive_bayes.MultinomialNB(alpha=0.68)
clfi.fit(X_tr, yi_tr)
predi = clfi.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yi_te, predi))

clfih = sklearn.naive_bayes.MultinomialNB(alpha=0.17)
clfih.fit(X_tr, yih_tr)
predih = clfih.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yih_te, predih))

0.9541666599125774
0.9804642870505454
0.9625897828905864
0.9766759168952911
0.9629835024869225
0.9670554537677515


In [219]:
Xwo_test = vectorizer.transform(test_data['comment_text'])
Xch_test = char_vectorizer.transform(test_data['comment_text'])
X_test = sp.sparse.hstack([Xwo_test, Xch_test])

In [221]:
clft.fit(X, yt)
clfst.fit(X, yst)
clfo.fit(X, yo)
clfth.fit(X, yth)
clfi.fit(X, yi)
clfih.fit(X, yih)

predt = clft.predict_proba(X_test)[:,1]
predst = clfst.predict_proba(X_test)[:,1]
predo = clfo.predict_proba(X_test)[:,1]
predth = clfth.predict_proba(X_test)[:,1]
predi = clfi.predict_proba(X_test)[:,1]
predih = clfih.predict_proba(X_test)[:,1]

In [222]:
sumbission_example['toxic']=predt
sumbission_example['severe_toxic']=predst
sumbission_example['obscene']=predo
sumbission_example['threat']=predth
sumbission_example['insult']=predi
sumbission_example['identity_hate']=predih

In [223]:
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000e+00,9.999463e-01,1.000000e+00,9.982259e-01,9.999999e-01,9.999888e-01
1,0000247867823ef7,8.936087e-04,1.001995e-05,4.225591e-04,9.153498e-07,2.509071e-04,3.806989e-05
2,00013b17ad220c46,3.540050e-03,2.164167e-05,9.529128e-04,1.131826e-07,6.313039e-04,5.822104e-05
3,00017563c3f7919a,1.371578e-05,4.579708e-09,1.934407e-06,4.860352e-10,2.025912e-06,4.068509e-09
4,00017695ad8997eb,2.314065e-02,4.318574e-05,8.149179e-03,1.257021e-05,6.962710e-03,1.612994e-04
5,0001ea8717f6de06,2.042284e-04,1.365516e-08,2.008960e-05,8.798229e-08,3.087423e-05,1.930190e-07
6,00024115d4cbde0f,2.840784e-06,2.915260e-10,6.463727e-07,2.546717e-11,4.388056e-07,1.192285e-09
7,000247e83dcc1211,9.215939e-01,3.482910e-02,5.986240e-01,2.065825e-02,6.662652e-01,7.374280e-02
8,00025358d4737918,2.241369e-04,2.106446e-09,1.935779e-05,1.486280e-08,6.155505e-06,1.755753e-07
9,00026d1092fe71cc,2.048622e-05,5.602823e-09,6.512067e-06,1.037708e-08,3.705293e-06,2.532325e-08


In [225]:
sumbission_example.to_csv('nb_tfidf1.csv', index=False)